# EURO STOXX50 - Base GBM Model

In this notebook, we detail our implementation of our base Geometric Brownian Motion model for the [Outperformance Bonus Certificate EURO STOXX 50](https://derivative.credit-suisse.com/ch/ch/en/detail/outperformance-bonus-certificate-euro-stoxx-50/CH1149494077/114949407), a derivative of the SX5E stock.

For our risk-neutral valuation, we perform the following:
1) Look back M time steps for historical data (given an M of 252, we'll take the subset of data from 11 August 2022 to 9 August 2023)
2) Estimate our model parameters ($\sigma = \sqrt{sample\ variance}$) from historical data for the GBM model)
3) Plug our estimated model parameters into our simulation functions and simulate N simulations from 10 August 2023 to our maturity date (e.g. monte carlo, monte carlo with antithetic variates)
4) Calculate our predicted stock price using the mean of the N simulations on our maturity date
5) Calculate our option price by pulling back from the predicted stock price

## Importing Data

In [1]:
# External Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy
from numba import jit, njit, prange
import time
import datetime
import warnings
import concurrent.futures
import multiprocessing
import ctypes

warnings.filterwarnings("ignore") # Suppress warnings from numba

# Our Code
from payoff_function import PayoffFunctions

We import our historical stock data, taken from [Yahoo Finance's STOXX50E](https://finance.yahoo.com/quote/%5ESTOXX50E/) into a Pandas dataframe called `S_all`:

In [2]:
DATA_FOLDER = "../Data"
S_all = pd.read_csv(f"{DATA_FOLDER}/STOXX50E_2Aug2022_13Nov2023.csv")
S_all = S_all[['Date', 'Adj Close']]

We then import the Euro area yields with 1-year maturity period, taken from the [European Central Bank data portal](https://www.ecb.europa.eu/stats/financial_markets_and_interest_rates/euro_area_yield_curves/html/index.en.html), as data for our risk-free interest rate:

In [3]:
ir = pd.read_csv(f"{DATA_FOLDER}/euro_area_1_year_yield_curve.csv").rename(columns={"DATE": "Date", "TIME PERIOD": "Date String", "Yield curve spot rate, 1-year maturity - Government bond, nominal, all issuers whose rating is triple A - Euro area (changing composition) (YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_1Y)": "Interest Rate"})
ir

,Date,Date String,Interest Rate
0,2004-09-06,06 Sep 2004,2.298838
1,2004-09-07,07 Sep 2004,2.328891
2,2004-09-08,08 Sep 2004,2.346666
3,2004-09-09,09 Sep 2004,2.308988
4,2004-09-10,10 Sep 2004,2.271566
...,...,...,...
4903,2023-11-07,07 Nov 2023,3.443486
4904,2023-11-08,08 Nov 2023,3.429405
4905,2023-11-09,09 Nov 2023,3.442951
4906,2023-11-10,10 Nov 2023,3.494435


Our timestep (dt) is defined as $\frac{1}{252}$, indicating 1 day out of 252 working days in a year.

In [4]:
dt = 1/252

We define our dates of interest as the dates between 9 Aug 2023 and 9 Nov 2023 (66 days in total):

In [5]:
actual_dates = ir[(ir['Date'] >= '2023-08-09') & (ir['Date'] <= '2023-11-09')]['Date'].values
print(f"Days in Total: {len(actual_dates)}")

Days in Total: 66


## Obtaining Model Parameters from Historical Data
For our base GBM model, we are only interested in $\sigma$, the sample variance of our historical data for each day.

For each day in the sliding window (from 9 Aug 2023 to 9 Nov 2023, 66 days in total), we look back M days and calculate $\sigma$. 

In [7]:
# Main Parameters
M = [21, 63, 252]

for M_value in M:
    sigma = np.zeros(len(actual_dates), dtype=np.float32)
    for i, date in enumerate(actual_dates):
        date_index = S_all.loc[S_all['Date'] == date].index[0]
        history_data = list(S_all.iloc[date_index-M_value:date_index, ]['Adj Close'])
        log_history_data = np.log(history_data)
        log_return = log_history_data[1:] - log_history_data[:-1]
        sigma[i] = np.std(log_return) / np.sqrt(dt)
    print(f"M: {M_value} - sigma ({actual_dates[0]}): {sigma[0]}, sigma ({actual_dates[-1]}): {sigma[-1]}, sigma (average): {sigma.mean()}")

M: 21 - sigma (2023-08-09): 0.1721991002559662, sigma (2023-11-09): 0.13019900023937225, sigma (average): 0.13611777126789093
M: 63 - sigma (2023-08-09): 0.1524956226348877, sigma (2023-11-09): 0.12539911270141602, sigma (average): 0.14484648406505585
M: 252 - sigma (2023-08-09): 0.1707075983285904, sigma (2023-11-09): 0.1513691544532776, sigma (average): 0.1620490998029709
